# Fire Crimes, Car Theft, and Burglary

In the following section we investigate the number of fire crimes, car theft and burglary per 100,000 residents by district per year.  

The goal for the temporal development of crime rates was not only to look at the overall crime rates, but also to show the trend for various types of crimes to identify noteworthy differences. However, creating and comparing 20 individual plots would have been quite a complex task.

To address this, the mean value of each crime feature for each district over the last 10 years was computed. Subsequently, a Principal Component Analysis (PCA) was conducted with these mean values, leading to the creation of a biplot. The biplot revealed that most crime features showed similarities, while car theft, fire crimes and burglary appeared to be distinct from the others, making them especially interesting for further investigation.

The plot for fire crimes is shown in {numref}`fig:crimes-fire`. We cannot make out a clear pattern.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from IPython import display
import time
import warnings

warnings.filterwarnings("ignore")

set1 = sns.color_palette("Set1")
tab10 = sns.color_palette("tab10")
qualitative_palette = sns.color_palette([set1[0], "#000099", set1[4], "#ff66ff", "#006600", "#ffcc00", "#000000", tab10[9], tab10[5], set1[8], "#00cc00", "#660066"])

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import warnings

warnings.filterwarnings("ignore")

bezirkskriminalitaet = pd.read_pickle("../data/08_kriminalitaet_auf_bezirksebene_2013_bis_2022.pkl")
bezirkskriminalitaet['BEZ'] = bezirkskriminalitaet['LOR-Schlüssel (Bezirksregion)'].astype(str).str[:2]

bezirkskriminalitaet = bezirkskriminalitaet.rename(columns={"Straftaten \n-insgesamt-": "Overall crime rate",
                                                            "Branddelikte \n-insgesamt-": "Fire crimes",
                                                            "Diebstahl von Kraftwagen": "Car theft",
                                                            "Wohnraum-\neinbruch": "Burglary"})

# Create a list of features to iterate over
features_list = ["Overall crime rate", "Fire crimes", "Car theft", "Burglary"]

# Create the categorical color palette with unique colors for each category
categories = bezirkskriminalitaet["Bezeichnung (Bezirksregion)"].unique()
num_categories = len(categories)

# Create the subplots (4 rows, 1 column)
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(10, 32), sharex=True, gridspec_kw={'top': 0.95})

# Iterate over the features and plot each on its respective subplot
for idx, feature in enumerate(features_list):
    # Convert the current feature column to integers
    bezirkskriminalitaet[feature] = bezirkskriminalitaet[feature].astype(int)
    
    # Group the data by "Bezeichnung (Bezirksregion)" and "Year" and sum the values for the current feature
    grouped_data = bezirkskriminalitaet.groupby(["Bezeichnung (Bezirksregion)", "Year"])[feature].sum().reset_index()

    # Select the current subplot
    ax = axes[idx]
    
    # Iterate over the unique values in the "Bezeichnung (Bezirksregion)" column
    for category, color in zip(categories, qualitative_palette):
        # Filter the data for the current category
        category_data = grouped_data[grouped_data["Bezeichnung (Bezirksregion)"] == category]

        # Plot the line for the current category on the current subplot
        ax.plot(category_data["Year"], category_data[feature], label=category, color=color, linewidth=3)

    # Set the subplot title, labels, and legend
    ax.set_title(f"{feature}", fontsize=15)
    ax.set_ylabel(f"{feature} per 100,000 residents", fontsize=12)
    ax.legend(title="Districts", loc="upper right", fontsize=8)

    # Get all unique years in the data for the current subplot
    unique_years = bezirkskriminalitaet["Year"].unique()

    # Set the x-ticks to be all unique years for the current subplot
    ax.set_xticks(unique_years)

    ax.tick_params(axis='both', which='both', labelbottom=True)

    # Disable scientific notation on the y-axis
    ax.ticklabel_format(style='plain')

    # Set the background color to gray
    ax.set_facecolor('lightgray')
    # Add a grid
    ax.grid(color='white', linestyle='-', linewidth=0.5)

plt.suptitle('Temporal development (2013-2022) of numbers per 100,000 residents \nof different kinds of crimes split by Berlin districts', fontsize=20)

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


```{figure} plots/fire_crimes.svg
---
width: 100%
name: fig:crimes-fire
---
Number of fire crimes per 100,000 residents in the years 2013–2022 by district.
```
The numbers for each district display significant variations. For instance, Treptow-Köpenick had the third-highest number of fire crimes in 2019 but the lowest in 2020, indicating a weak or non-existent correlation between fire crimes and other indicators.

Looking at the car thefts in {numref}`fig:crimes-car`, we see significant fluctuations from year to year.

```{figure} plots/car_theft.svg
---
width: 100%
name: fig:crimes-car
---
Number of car thefts per 100,000 residents in the years 2013–2022 by district.
```
These fluctuations differ from the overall crime rate in {numref}`fig:crimes-overall`. Notably, Lichtenberg, Marzahn-Hellersdorf, Treptow-Köpenick, and Charlottenburg-Wilmersdorf have the highest car theft numbers, despite being among the lowest for overall crime (except for Charlottenburg-Wilmersdorf).  
A possible interpretation is that these districts are perceived as safer, with residents having higher incomes and possessing more valuable cars. This makes them attractive targets for car thieves.
Additionally, it is observed that the car theft rate is notably low for all districts during 2020 and 2021, the years with the heaviest Covid restrictions, increases again in 2022.

The number of burglaries is shown in {numref}`fig:crimes-burglary`. The numbers for all districts appear relatively similar, particularly since 2016.

```{figure} plots/burglary.svg
---
width: 100%
name: fig:crimes-burglary
---
Number of burglaries per 100,000 residents in the years 2013–2022 by district.
```
Notably, Steglitz-Zehlendorf, known for having the lowest overall crime rate, exhibited a relatively high burglary rate along with Charlottenburg-Wilmersdorf. The most remarkable aspect of this plot was the general declining trend in burglary numbers over the years.